<a href="https://colab.research.google.com/github/Reinaldo-34/sentimentGoogle/blob/main/googleReadV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## import of all relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [4]:
## open the google sheet which has the data
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1_vROEMAQ29haCkI1yowbZh1h5qJfFWctnOqhxHneytE/edit#gid=0')

import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
sheet = wb.worksheet('Sheet1')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
reviewList = list(df['Review text'])

In [6]:
## iterate through all items in ReviewList and tokenize
tokens = []

for i in range(len(reviewList)):
  x = nltk.word_tokenize(reviewList[i])
  tokens.append(x)

In [7]:
## VaderSentiment analyzer - assign a sentiment score to each review
score = np.transpose([]) 
analyzer = SentimentIntensityAnalyzer()
for sentence in reviewList:
  vs = analyzer.polarity_scores(sentence)
  score = np.append(score,vs['compound'])
##print("{:-<65} {}".format(sentence, str(vs)))

df['Score'] = score

print(score)
score.size 

[ 0.9598  0.981   0.9856  0.5413  0.854  -0.9044  0.7712  0.8622  0.9611
  0.9516  0.7841  0.8955  0.9782  0.4408  0.4927  0.9523  0.9741  0.659
  0.9488  0.8261  0.9439  0.4025  0.7351  0.9694  0.9677  0.8221  0.8817
  0.9775  0.2152  0.9866  0.9022  0.9035  0.9966  0.8797  0.9416  0.9183
 -0.3766  0.9501  0.9022  0.9651  0.836   0.9571 -0.9514  0.6124  0.984
  0.8999  0.886   0.9825  0.946   0.9413  0.9846  0.9545  0.9022 -0.9125
  0.862   0.6778  0.8517 -0.2263  0.0258  0.4754  0.5719  0.6705  0.8221]


63

In [8]:
## classifying compound scores into positive, negative or neutral
def get_class(score):
  if score >= 0.5:
    return 'Positive'
  elif score <= -0.5:
    return 'Negative'
  else:
    return 'Neutral'

## making the sentiment scores into its own dataframe column
df['Sentiment'] = df['Score'].apply(get_class)

In [9]:
## populating a list of consultants
consultant = ['Emmanuel', 'Jack', 'Kevin', 'Lisa', 'Mohit', 'Rebekah', 'Reneh', 'Rossetta', 'Sophie', 'Terry']

## creating an empty list 
owner = []
for k in range(63):     ##iterate through the entire dataset
  x = 0
  for i in tokens[k]:   ##iterate through all tokens
    for j in consultant: ##iterate through all consultants
      if i == j:        ##find matching consultant items in the tokens
        x = x+1 
        y = j
  if x > 0:
    owner.append(y)
  else:
    owner.append("STPG")

df['Owner'] = owner

In [10]:
#create new dataframe schema which includes the columns we want
newest_df = {
  ##  'Review Text' : df['Review text'].astype(str),
    'Rating' : df['Rating'].astype(int),
    'Score' : df['Score'].astype(float),
    'Sentiment' : df['Sentiment'].astype(str),
    'Owner' : df['Owner'].astype(str)
}

newDataFrame = pd.DataFrame(newest_df)

In [11]:
print(newDataFrame)

    Rating   Score Sentiment     Owner
1        5  0.9598  Positive  Emmanuel
2        5  0.9810  Positive  Emmanuel
3        5  0.9856  Positive      STPG
4        5  0.5413  Positive  Emmanuel
5        5  0.8540  Positive  Emmanuel
..     ...     ...       ...       ...
59       1  0.0258   Neutral      STPG
60       5  0.4754   Neutral      STPG
61       5  0.5719  Positive      STPG
62       5  0.6705  Positive  Emmanuel
63       5  0.8221  Positive      STPG

[63 rows x 4 columns]
